In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

In [3]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
highway_skims_dir = _join(skims_dir, "highway")

In [4]:
# MD hghway Skim
hwy_skim = omx.open_file(_join(highway_skims_dir, 'HWYSKMMD.omx'))

In [8]:
tt_time = skim_core_to_df(hwy_skim, 'TIMEDAM', cols =['orig', 'dest', 'hwy_time'])

In [9]:
tt_time

,orig,dest,hwy_time
0,1,1,0.872216
1,2,1,14.793816
2,3,1,19.199364
3,4,1,17.941429
4,5,1,17.207962
...,...,...,...
11242604,3349,3353,112.631569
11242605,3350,3353,115.529465
11242606,3351,3353,88.495483
11242607,3352,3353,78.107613


In [10]:
major_centers = pd.read_csv(_join(params['common_dir'], 'emp_centers.csv'))

In [18]:
major_centers

,taz,emp,hubs
0,1,0,0
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0
...,...,...,...
3327,3328,0,0
3328,3329,0,0
3329,3330,0,0
3330,3331,0,0


In [21]:
tazs = list(major_centers.loc[major_centers['emp'] > 0]['taz'])

In [22]:
tazs

[17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41]

In [23]:
tt_od = tt_time[tt_time['orig'].isin(tazs) & (tt_time['dest'].isin(tazs))]

In [24]:
tt_od

,orig,dest,hwy_time
53664,17,17,0.285261
53665,18,17,1.205978
53666,19,17,1.210511
53667,20,17,1.794721
53668,21,17,5.842018
...,...,...,...
134156,37,41,6.369782
134157,38,41,5.209092
134158,39,41,5.353117
134159,40,41,1.119378


In [ ]:
# actual travel time savings between key ODs